<a href="https://colab.research.google.com/github/theforces/Error-level-analysis/blob/main/ECISV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Create folder for images
# https://stackoverflow.com/questions/60706752/how-to-make-directory-on-the-google-colab

import os.path
from os import path

if path.exists('/content/images') == False:
  os.mkdir('/content/images')

os.chdir('/content/images')
!pwd
!ls

/content/images


In [ ]:
#@title Upload files function
# https://stackoverflow.com/questions/49186905/loading-images-in-google-colab

def upload_files():
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(k, 'wb').write(v)
  return list(uploaded.keys())

In [ ]:
upload_files()

In [ ]:
apply_blur = True #@param {type:"boolean"}

apply_invert = True #@param {type:"boolean"}

download_blurred_or_inverted_image = False #@param {type:"boolean"}

In [ ]:
import ipywidgets as widgets
slider = widgets.IntSlider(value=5, max=100)
display(slider)

IntSlider(value=5)

In [ ]:
jpeg_compression_rate = slider.value
print(jpeg_compression_rate)

90


In [ ]:
#@title Process one or more images


import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import math


# https://www.pyimagesearch.com/2015/10/05/opencv-gamma-correction/
def adjust_gamma(image, gamma=1.0):  
	# build a lookup table mapping the pixel values [0, 255] to
	# their adjusted gamma values
	invGamma = 1.0 / gamma
	table = np.array([((i / 255.0) ** invGamma) * 255
		for i in np.arange(0, 256)]).astype("uint8")
	# apply gamma correction using the lookup table
	return cv2.LUT(image, table)

for root, dirs, files in os.walk('/content/images/'):
    for file in files:
        filename, extension = os.path.splitext(file)
        if extension == '.jpg' and '_c' not in filename:
          
            im = cv2.imread(file)
            original_im = im.copy()
            #im_resized = cv2.resize(im, (224, 224), interpolation=cv2.INTER_LINEAR)

            if apply_blur:
              im =  cv2.GaussianBlur(im,(7,7),0)
              copy_blurred_image = im.copy()
              if download_blurred_or_inverted_image:
                cv2.imwrite(filename + '_blurred' + extension,im)

            compressed_image = filename + '_c' + str(jpeg_compression_rate) + extension
            cv2.imwrite(compressed_image, im, [cv2.IMWRITE_JPEG_QUALITY, jpeg_compression_rate])
            comp_im = cv2.imread(compressed_image)
            diff = cv2.absdiff(im, comp_im)

            diff = np.asarray(diff)
            max_diff = np.amax(diff)
            
            scale = 255.0 / max_diff
            diff = adjust_gamma(diff, scale)

            if apply_invert:
              diff = cv2.bitwise_not(diff) #https://stackoverflow.com/questions/19580102/inverting-image-in-python-with-opencv
              copy_inverted_diff_image = diff.copy()
              if download_blurred_or_inverted_image:
                cv2.imwrite(filename + '_inverted' + extension,diff)

            if not apply_blur and not apply_invert:
              fig = plt.figure(figsize=(20, 20))

              ax1 = fig.add_subplot(1,2,1)
              ax1.imshow(cv2.cvtColor(original_im, cv2.COLOR_BGR2RGB))
              ax1.title.set_text("Analysed image is:" + filename)

              ax2 = fig.add_subplot(1,2,2)
              ax2.imshow(diff)
              ax2.title.set_text("Difference for:" + filename + ".Compressed image used:" + compressed_image)

              plt.show()


            if apply_blur and apply_invert == False:
              fig = plt.figure(figsize=(20, 20))

              ax1 = fig.add_subplot(1,3,1)
              ax1.imshow(cv2.cvtColor(original_im, cv2.COLOR_BGR2RGB))
              ax1.title.set_text("Analysed image is:" + filename)

              ax2 = fig.add_subplot(1,3,2)
              ax2.imshow(cv2.cvtColor(copy_blurred_image, cv2.COLOR_BGR2RGB))
              ax2.title.set_text("Blurred image for:" + filename)

              ax3 = fig.add_subplot(1,3,3)
              ax3.imshow(diff)
              ax3.title.set_text("Difference for:" + filename + ".Compressed image used:" + compressed_image)

              plt.show()


            if apply_invert and apply_blur == False:
              fig = plt.figure(figsize=(20, 20))

              ax1 = fig.add_subplot(1,3,1)
              ax1.imshow(cv2.cvtColor(original_im, cv2.COLOR_BGR2RGB))
              ax1.title.set_text("Analysed image is:" + filename)

              ax2 = fig.add_subplot(1,3,2)
              ax2.imshow(cv2.cvtColor(copy_inverted_diff_image, cv2.COLOR_BGR2RGB))
              ax2.title.set_text("Inverted image for:" + filename)

              ax3 = fig.add_subplot(1,3,3)
              ax3.imshow(diff)
              ax3.title.set_text("Difference for:" + filename + ".Compressed image used:" + compressed_image)

              plt.show()


            if apply_blur and apply_invert:
              fig = plt.figure(figsize=(20, 20))

              ax1 = fig.add_subplot(2,2,1)
              ax1.imshow(cv2.cvtColor(original_im, cv2.COLOR_BGR2RGB))
              ax1.title.set_text("Analysed image is:" + filename)

              ax2 = fig.add_subplot(2,2,2)
              ax2.imshow(cv2.cvtColor(copy_blurred_image, cv2.COLOR_BGR2RGB))
              ax2.title.set_text("Blurred image for:" + filename)

              ax3 = fig.add_subplot(2,2,3)
              ax3.imshow(cv2.cvtColor(copy_inverted_diff_image, cv2.COLOR_BGR2RGB))
              ax3.title.set_text("Inverted image for:" + filename)

              ax4 = fig.add_subplot(2,2,4)
              ax4.imshow(copy_inverted_diff_image)
              ax4.title.set_text("Difference for:" + filename + ".Compressed image used:" + compressed_image)

              plt.show()

            print('\n')        


In [ ]:
#@title Archive image results
# https://stackoverflow.com/questions/50453428/how-do-i-download-multiple-files-or-an-entire-folder-from-google-colab

!zip -r /content/results.zip /content/images/

In [ ]:
#@title Download results archive
from google.colab import files
files.download("/content/results.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>